# Sea Surface Temperature

In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect
from regions import boolean_mask, SOM_area, Nino12, Nino34, global_ocean,\
                    AMO_area, T_exT_area, gl_ocean_rect, NPacific_mask_rect
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA
from xr_regression import xr_linear_trends_2D, xr_linear_trend, ocn_field_regression

# global yearly avg. SST map and trends

In [ ]:
MASK = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).REGION_MASK

In [ ]:
# %%time
# # ca. 4:30 min
# # stacking files into one xr Dataset object
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM('ocn', run, 'yrly', name='TEMP_PD')):
#         print(y)
#         da = xr.open_dataset(s, decode_times=False).TEMP[0,:,:]
#         da = da.drop(['z_t', 'ULONG', 'ULAT'])
#         da['TLAT' ] = da['TLAT' ].round(decimals=2)
#         da['TLONG'] = da['TLONG'].round(decimals=2)
#         del da.encoding["contiguous"]
#         ds = t2ds(da=da, name='SST', t=int(round(da.time.item())))
#         ds.to_netcdf(path=f'{path_samoc}/SST/SST_yrly_{run}_{y}.nc', mode='w')
     
#     combined = xr.open_mfdataset(f'{path_samoc}/SST/SST_yrly_{run}_*.nc',
#                                  concat_dim='time',
#                                  autoclose=True,
#                                  coords='minimal')
#     combined.to_netcdf(f'{path_samoc}/SST/SST_yrly_{run}.nc')
#     # remove extra netCDF files

CPU times: user 1min 25s, sys: 3min 16s, total: 4min 41s
Wall time: 11min 18s

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc')
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' )

In [ ]:
xa = SST_yrly_ctrl.mean(dim='time').where(MASK>0)

In [ ]:
fn    = f'{path_results}/SST/SST_ctrl_mean'
cm    = discrete_cmap(17, cmocean.cm.thermal)
f, ax = map_robinson(xa=xa, domain='ocn_U', cmap=cm, minv=-2, maxv=32, label='SST [$^\circ$C]', filename=fn)

In [ ]:
%%time
SST_trend_ctrl = ocn_field_regression(SST_yrly_ctrl)
SST_trend_rcp  = ocn_field_regression(SST_yrly_rcp )

In [ ]:
cmap  = discrete_cmap(12, shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk'))
label = 'SST trend [K/century]'

xa = SST_trend_ctrl*100
fn = f'{path_results}/SST/SST_trend_ctrl'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

xa = SST_trend_rcp*100
fn = f'{path_results}/SST/SST_trend_rcp'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

### hiatus trend

In [ ]:
SST_yrly_rcp[20:29,:,:]

In [ ]:
SST_trend_rcp_hiatus  = ocn_field_regression(SST_yrly_rcp[20:29,:,:])

In [ ]:
fn = f'{path_results}/SST/SST_trend_rcp_hiatus'
cmap = shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk')
label = 'SST trend [K/century]'
xa = SST_trend_rcp_hiatus*100
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-4, maxv=8, label=label, filename=fn)

# SST indices

In [ ]:
MASK_ocn  = boolean_mask('ocn'     , 0)
MASK_rect = boolean_mask('ocn_rect', 0)

In [ ]:
TAREA       = xr_AREA('ocn')
AREA_rect   = xr_AREA('ocn_rect')

In [ ]:
global_area = TAREA.where(MASK_ocn).sel(global_ocean).sum()
print(f'{global_area.item():e}')
global_area2 = AREA_rect.where(MASK_rect).sel(gl_ocean_rect).sum()
print(f'{global_area2.item():e}')

# Southern Ocean

## Southern Ocean Mode index
annual SST anomaly (50S-35S, 0E-50W), here detrended

In [ ]:
# %%time
# # 30 sec
# SOMi_area = TAREA.sel(SOM_area).sum()
# SOMi_ctrl = SST_index(xa_SST=SST_yrly_ctrl, AREA=TAREA, index_loc=SOM_area, AREA_index=SOMi_area, MASK=MASK_ocn, dims=('nlat', 'nlon'))
# SOMi_rcp  = SST_index(xa_SST=SST_yrly_rcp , AREA=TAREA, index_loc=SOM_area, AREA_index=SOMi_area, MASK=MASK_ocn, dims=('nlat', 'nlon'))

In [ ]:
# SOMi_trend_ctrl = np.polyfit(SOMi_ctrl.time, SOMi_ctrl, 1)
# SOMi_trend_rcp  = np.polyfit(SOMi_rcp .time, SOMi_rcp , 1)

# SOMi_detr_ctrl = SOMi_ctrl - (SOMi_trend_ctrl[1] + SOMi_trend_ctrl[0]*SOMi_ctrl.time)
# SOMi_detr_rcp  = SOMi_rcp  - (SOMi_trend_rcp[1]  + SOMi_trend_rcp[0] *SOMi_rcp .time)

# SOMi_detr_ctrl.to_netcdf(f'{path_results}/SST/SOM_index_ctrl.nc')
# SOMi_detr_rcp .to_netcdf(f'{path_results}/SST/SOM_index_rcp.nc' )

In [ ]:
SOMi_detr_ctrl = xr.open_dataarray(f'{path_results}/SST/SOM_index_ctrl.nc', decode_times=False)
SOMi_detr_rcp  = xr.open_dataarray(f'{path_results}/SST/SOM_index_rcp.nc' , decode_times=False)

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

# plt.axhline(0,c='k', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_ctrl, c='C0', lw=.5)
plt.plot(SOMi_rcp .time/365-1800, SOMi_rcp , c='C1', lw=.5)

plt.plot(SOMi_ctrl.time/365     , SOMi_trend_ctrl[1] + SOMi_trend_ctrl[0]*SOMi_ctrl.time, ls='--')
plt.plot(SOMi_rcp .time/365-1800, SOMi_trend_rcp [1] + SOMi_trend_rcp [0]*SOMi_rcp.time , ls='--')

L1, = plt.plot(SOMi_ctrl.time/365     , SOMi_ctrl.rolling({'time':5}, center=True).mean(), c='C0', lw=2, label='CTRL')
L2, = plt.plot(SOMi_rcp .time/365-1800, SOMi_rcp .rolling({'time':5}, center=True).mean(), c='C1', lw=2, label='RCP')

plt.text(.02,.95, r'[35,50$^\circ$S]$\times$[-50,0$^\circ$E]', ha='left',transform=ax.transAxes, fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('raw SOM index [K]', fontsize=16)
plt.legend(fontsize=16, ncol=2, loc=4, handles=[L1, L2])
plt.savefig(f'{path_results}/SST/SOM_raw')

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

plt.axhline(0,c='k', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_detr_ctrl, c='C0', lw=.5)
plt.plot(SOMi_rcp .time/365-1800, SOMi_detr_rcp , c='C1', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_detr_ctrl.rolling({'time':5}, center=True).mean(), c='C0', lw=2, label='CTRL')
plt.plot(SOMi_rcp .time/365-1800, SOMi_detr_rcp .rolling({'time':5}, center=True).mean(), c='C1', lw=2, label='RCP')

plt.text(.02,.95, r'[35,50$^\circ$S]$\times$[-50,0$^\circ$E]', ha='left',transform=ax.transAxes, fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('SOM index [K]', fontsize=16)
plt.legend(fontsize=16, ncol=2)
plt.savefig(f'{path_results}/SST/SOM_index')

# Pacific Ocean

## Interdecadal Pacific Oscillation
Karl et al

## Pacific Decadal Oscillation
> The leading EOF of monthlySST anomalies over the North Pacific (after removing the global mean SST anomaly) and its associated PC time series are termed the Pacific Decadal Oscillation (PDO)

1. compute monthly global mean SST

2. create North Pacific monthly output fields
    2.1. determine extend of grid
    2.2. reduce all coordinates appropriately
    2.3. loop through all months and save as single netcdf

3. EOF analysis

maybe use ocn rect for it

In [ ]:
# %%time
# # 29 min
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run=run, tavg='monthly')):
#         SST = xr.open_dataset(s, decode_times=False).TEMP[0,0,:,:]
#         SST_gm = SST_index(xa_SST=SST, AREA=TAREA, index_loc=global_ocean, AREA_index=global_area, MASK=MASK_ocn, dims=('nlat', 'nlon'))
#         if m==1: print(y, SST_gm.item()) 
#         if i==0:  SST_new = SST_gm
#         else:     SST_new = xr.concat([SST_new, SST_gm], dim='time')
#     SST_new.to_netcdf(f'{path_results}/SST_global_mean_monthly_{run}.nc')

In [ ]:
# %%time
# # 11 min
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn_rect', run=run, tavg='monthly')):
#         SST = xr.open_dataset(s, decode_times=False).TEMP[0,:,:]
#         SST_gm = SST_index(xa_SST=SST, AREA=AREA_rect, index_loc=gl_ocean_rect, AREA_index=global_area2, MASK=MASK_rect, dims=('t_lat', 't_lon'))
#         if m==1: print(y, SST_gm.item()) 
#         if i==0:  SST_new = SST_gm
#         else:     SST_new = xr.concat([SST_new, SST_gm], dim='time')
#     SST_new.to_netcdf(f'{path_results}/SST_global_mean_monthly_rect_{run}.nc')

In [ ]:
SST_gm_ctrl      = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_ctrl.nc'     , decode_times=False)
SST_gm_rcp       = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rcp.nc'      , decode_times=False)
SST_gm_rect_ctrl = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_ctrl.nc', decode_times=False)
SST_gm_rect_rcp  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_rcp.nc' , decode_times=False)

In [ ]:
plt.plot(SST_gm_ctrl)
plt.plot(SST_gm_rcp )
plt.plot(SST_gm_rect_ctrl)
plt.plot(SST_gm_rect_rcp )

In [ ]:
plt.axhline(0, c='k', lw=.5)
plt.plot(SST_gm_rect_ctrl.time/12+100, SST_gm_rect_ctrl-SST_gm_ctrl[:len(SST_gm_rect_ctrl)])
plt.plot(SST_gm_rect_rcp .time/12+300, SST_gm_rect_rcp -SST_gm_rcp [:len(SST_gm_rect_rcp )])

somehow there is an almost constant low bias in the global mean SST when calculatd in the rect files

In [ ]:
NPac_MASK = NPacific_mask_rect()
NPac_area = xr_AREA('ocn_rect').where(NPac_MASK, drop=True)

In [ ]:
# %%time
# # 51 mins for 162 years of ctrl
# # 9 min for 67 years of rcp
# 
# for j, run in enumerate(['ctrl', 'rcp']):
#     SST_gm_rect = [SST_gm_rect_ctrl, SST_gm_rect_rcp][j]
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn_rect', tavg='monthly', run=run)):
#         if m==1:
#             print(y)
#         xa = xr.open_dataset(s, decode_times=False).TEMP[0,:,:].where(NPac_MASK, drop=True) - SST_gm_rect[i]
#         if i==0:
#             xa_out = xa.copy()
#         else:
#             xa_out = xr.concat([xa_out, xa], dim='time')
#         xa_out.to_netcdf(f'{path_samoc}/SST/NPacific_anomaly_monthly_rect_{run}.nc')

In [ ]:
SST_NPac_rect_ctrl = xr.open_dataarray(f'{path_samoc}/SST/NPacific_anomaly_monthly_rect_ctrl.nc', decode_times=False)
SST_NPac_rect_rcp  = xr.open_dataarray(f'{path_samoc}/SST/NPacific_anomaly_monthly_rect_rcp.nc' , decode_times=False)

In [ ]:
# %%time
# # 30 sec for rcp
# # 2 mins for ctrl
# fn = f'{path_results}/SST/NPac_ctrl.nc'
# eof_ctrl, pc_ctrl = EOF_SST_analysis(xa=SST_NPac_rect_ctrl, weights=NPac_area, fn=fn)
# fn = f'{path_results}/SST/NPac_rcp.nc'
# eof_rcp , pc_rcp  = EOF_SST_analysis(xa=SST_NPac_rect_rcp , weights=NPac_area, fn=fn)

In [ ]:
NPac_ctrl = xr.open_dataset(f'{path_results}/SST/NPac_ctrl.nc', decode_times=False)
NPac_rcp  = xr.open_dataset(f'{path_results}/SST/NPac_rcp.nc' , decode_times=False)

In [ ]:
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(NPac_ctrl.time/12+100, NPac_ctrl.pcs.rolling(time= 1, center=True).mean(), c='C0', lw=1)
plt.plot(NPac_rcp .time/12+200, NPac_rcp .pcs.rolling(time= 1, center=True).mean(), c='C1', lw=1)
plt.plot(NPac_ctrl.time/12+100, NPac_ctrl.pcs.rolling(time=12, center=True).mean(), c='C0', lw=3)
plt.plot(NPac_rcp .time/12+200, NPac_rcp .pcs.rolling(time=12, center=True).mean(), c='C1', lw=3)
plt.xlabel('time [years]'             , fontsize=16)
plt.ylabel('first principal component', fontsize=16)

In [ ]:
f, ax = plt.subplots(1,1,figsize=(8,5))
ax.tick_params(labelsize=14)
ax.axhline(0, c='k', lw=.5)
ax.plot(NPac_ctrl.time/12+100, NPac_ctrl.pcs.rolling(time= 12, center=True).mean(), c='C0', lw=1)
ax.plot(NPac_rcp .time/12+200, NPac_rcp .pcs.rolling(time= 12, center=True).mean(), c='C1', lw=1)
ax.plot(NPac_ctrl.time/12+100, NPac_ctrl.pcs.rolling(time=120, center=True).mean(), c='C0', lw=3)
ax.plot(NPac_rcp .time/12+200, NPac_rcp .pcs.rolling(time=120, center=True).mean(), c='C1', lw=3)
ax.set_xlabel('time [years]'             , fontsize=16)
ax.set_ylabel('PDO', fontsize=16)
ax.text(.02, .02, '10 year running mean', transform=ax.transAxes, fontsize=14, color='grey')
plt.tight_layout()
plt.savefig(f'{path_results}/SST/PDO_index')

In [ ]:
xa = NPac_ctrl.eofs[0]
cmap = discrete_cmap(14,  shifted_color_map(cmocean.cm.balance, start=.5-1./12, midpoint=0.5, stop=1., name='shrunk'))
f = plt.figure(figsize=(8,5))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=190))
# ax.set_extent([.02,.98,.12,.86])
im1 = ax.pcolormesh(xa.t_lon, xa.t_lat, xa.values,
              cmap=cmap, vmin=-1, vmax=6,
                           transform=ccrs.PlateCarree() )
ax.add_feature(cartopy.feature.LAND, facecolor='grey', edgecolor='k', alpha=0.5)
cb = plt.colorbar(im1, orientation='horizontal')
cb.set_label('correlation coefficient', fontsize=12)
ax.set_title('EOF1 expressed as correlation', fontsize=16)
plt.savefig(f'{path_results}/SST/PDO_EOF', bbox_inches='tight')
# plt.tight_layout()
# (NPac_rcp.eofs-NPac_ctrl.eofs).plot()

## ENSO

In [ ]:
# %%time
# #5-10 sec per year
# Nino12_AREA = TAREA.sel(Nino12).sum(dim=('nlat','nlon'))
# Nino34_AREA = TAREA.sel(Nino34).sum(dim=('nlat','nlon'))
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run=run, tavg='monthly')):
#         if m==1: print(y)
#         SST = xr.open_dataset(s, decode_times=False).TEMP[0,0,:,:]
#         N12 = SST_index(xa_SST=SST, AREA=TAREA, index_loc=Nino12, AREA_index=Nino12_AREA, MASK=MASK_ocn, dims=('nlat', 'nlon'))
#         N34 = SST_index(xa_SST=SST, AREA=TAREA, index_loc=Nino34, AREA_index=Nino34_AREA, MASK=MASK_ocn, dims=('nlat', 'nlon'))
#         if i==0:
#             N12_new = N12
#             N34_new = N34
#         else:
#             N12_new = xr.concat([N12_new, N12], dim='time')
#             N34_new = xr.concat([N34_new, N34], dim='time')
#     N12_new.to_netcdf(path=f'{path_samoc}/SST/Nino12_{run}.nc')
#     N34_new.to_netcdf(path=f'{path_samoc}/SST/Nino34_{run}.nc')

In [ ]:
N12_ctrl = xr.open_dataarray(f'{path_samoc}/SST/Nino12_ctrl.nc', decode_times=False)
N34_ctrl = xr.open_dataarray(f'{path_samoc}/SST/Nino34_ctrl.nc', decode_times=False)
N12_rcp  = xr.open_dataarray(f'{path_samoc}/SST/Nino12_rcp.nc' , decode_times=False)
N34_rcp  = xr.open_dataarray(f'{path_samoc}/SST/Nino34_rcp.nc' , decode_times=False)

In [ ]:
# linear trend
N12_ctrl_lp = np.polyfit(N12_ctrl.time, N12_ctrl.values, 1)
N12_lf_ctrl = N12_ctrl_lp[0]*N12_ctrl.time + N12_ctrl_lp[1]
N34_ctrl_lp = np.polyfit(N34_ctrl.time, N34_ctrl.values, 1)
N34_lf_ctrl = N34_ctrl_lp[0]*N34_ctrl.time + N34_ctrl_lp[1]
N12_rcp_lp  = np.polyfit(N12_rcp.time, N12_rcp.values, 1)
N12_lf_rcp  = N12_rcp_lp[0]*N12_rcp.time + N12_rcp_lp[1]
N34_rcp_lp  = np.polyfit(N34_rcp.time, N34_rcp.values, 1)
N34_lf_rcp  = N34_rcp_lp[0]*N34_rcp.time + N34_rcp_lp[1]
# seasonal cycle

In [ ]:
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)

L1, = plt.plot(N12_ctrl.time/365, N12_ctrl, c='C0', ls='-' ,  alpha=.5, label='CTRL Nino12')
plt.plot(N12_ctrl.time/365, N12_lf_ctrl   , c='C0', ls='--', lw=2 )
L2, = plt.plot(N34_ctrl.time/365, N34_ctrl, c='C2', ls='-' ,  alpha=.5, label='CTRL Nino34')
plt.plot(N12_ctrl.time/365, N34_lf_ctrl   , c='C2', ls='--', lw=2 )
plt.axvline(200, c='g', lw=.5)
L3, = plt.plot(N12_rcp.time/365-1700, N12_rcp, c='C1', ls='-' , alpha=.5, label='RCP Nino12')
plt.plot(N12_rcp.time/365-1700, N12_lf_rcp   , c='C1', ls='--', lw=2 )
L4, = plt.plot(N34_rcp.time/365-1700, N34_rcp, c='C3', ls='-' , alpha=.5, label='RCP Nino34')
plt.plot(N34_rcp.time/365-1700, N34_lf_rcp   , c='C3', ls='--', lw=2 )
plt.ylabel('monthly temperature [$^\circ$C]', fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.tight_layout()
plt.legend(handles=[L2, L1, L4, L3], ncol=2, fontsize=14, loc=9, frameon=False)
plt.savefig(f'{path_results}/SST/ENSO_raw')

In [ ]:
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)


N12_mean_ctrl = np.empty((12))
N34_mean_ctrl = np.empty((12))
N12_mean_rcp  = np.empty((12))
N34_mean_rcp = np.empty((12))

plt.axhline(0, c='k', lw=.5)

for i in range(12):
    N12_mean_ctrl[i] = (N12_ctrl-N12_lf_ctrl)[i::12].mean()
    N12_mean_rcp [i] = (N12_rcp -N12_lf_rcp )[i::12].mean()
    N12_std_ctrl     = (N12_ctrl-N12_lf_ctrl)[i::12].std()
    N12_std_rcp      = (N12_rcp -N12_lf_rcp )[i::12].std()
    
    N34_mean_ctrl[i] = (N34_ctrl-N34_lf_ctrl)[i::12].mean()
    N34_mean_rcp [i] = (N34_rcp -N34_lf_rcp )[i::12].mean()
    N34_std_ctrl     = (N34_ctrl-N34_lf_ctrl)[i::12].std()
    N34_std_rcp      = (N34_rcp -N34_lf_rcp )[i::12].std()    
    
    S1 = plt.scatter([i+0.7], N12_mean_ctrl[i], c='C0', label='N12 CTRL')
    plt.plot(2* [i+0.7], [N12_mean_ctrl[i]-N12_std_ctrl, N12_mean_ctrl[i]+N12_std_ctrl], c='C0')
    S2 = plt.scatter([i+0.9], N12_mean_rcp [i], c='C1', label='N12 RCP')
    plt.plot(2* [i+0.9], [N12_mean_rcp [i]-N12_std_rcp , N12_mean_rcp[i] +N12_std_rcp], c='C1')
    
    S3 = plt.scatter([i+1.1], N34_mean_ctrl[i], c='C0', marker='D', label='N34 CTRL')
    plt.plot(2* [i+1.1], [N34_mean_ctrl[i]-N34_std_ctrl, N34_mean_ctrl[i]+N34_std_ctrl], c='C0', ls=':')
    S4 = plt.scatter([i+1.3], N34_mean_rcp [i], c='C1', marker='D', label='N34 RCP')
    plt.plot(2* [i+1.3], [N34_mean_rcp [i]-N34_std_rcp , N34_mean_rcp[i] +N34_std_rcp ], c='C1', ls=':')

plt.ylabel('temperature anomaly [K]', fontsize=16)
plt.xlabel('month', fontsize=16)
plt.xticks(np.arange(1,13), fontsize=14)
plt.legend(handles=[S1, S2, S3, S4], ncol=2)
plt.tight_layout()
plt.savefig(f'{path_results}/SST/ENSO_seasonal_cycle')

Claudia suggested to remove the seasonal signal (averages of all the Januaries, Februaries, ...). First I will remove a linear trend from both the CTRL and RCP indices.

In [ ]:
N12_detr_des_ctrl = (N12_ctrl-N12_lf_ctrl).copy()
N34_detr_des_ctrl = (N34_ctrl-N34_lf_ctrl).copy()
N12_detr_des_rcp  = (N12_rcp -N12_lf_rcp ).copy()
N34_detr_des_rcp  = (N34_rcp -N34_lf_rcp ).copy()
for i in range(12):
    N12_detr_des_ctrl[i::12] = N12_detr_des_ctrl[i::12]-N12_mean_ctrl[i]
    N34_detr_des_ctrl[i::12] = N34_detr_des_ctrl[i::12]-N34_mean_ctrl[i]
    N12_detr_des_rcp [i::12] = N12_detr_des_rcp [i::12]-N12_mean_rcp [i]
    N34_detr_des_rcp [i::12] = N34_detr_des_rcp [i::12]-N34_mean_rcp [i]

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(12,8), sharex=True)
for i in range(2):
    ax[i].tick_params(labelsize=14)

ax[0].axhline(0, c='k', lw=.5)
ax[0].axvline(200, c='g', lw=.5)
ax[0].plot(N12_detr_des_ctrl.time/365    , N12_detr_des_ctrl.rolling({'time':1}, center=True).mean() )
ax[0].plot(N12_detr_des_rcp.time/365-1700, N12_detr_des_rcp .rolling({'time':1}, center=True).mean() )

ax[1].axhline(0, c='k', lw=.5)
ax[1].axvline(200, c='g', lw=.5)
ax[1].plot(N34_detr_des_ctrl.time/365    , N34_detr_des_ctrl.rolling({'time':1}, center=True).mean() )
ax[1].plot(N34_detr_des_rcp.time/365-1700, N34_detr_des_rcp .rolling({'time':1}, center=True).mean() )
ax[1].set_xticks(np.arange(100,400,20))

ax[1].set_xlabel('time [years]'     , fontsize=16)
ax[0].set_ylabel('Nino1+2 index [K]', fontsize=16)
ax[1].set_ylabel('Nino3.4 index [K]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/SST/ENSO_indices')

In [ ]:
# correlation between indices
plt.figure()
plt.axhline(0, c='k', lw=.5)
plt.axvline(0, c='k', lw=.5)
plt.scatter(N12_detr_des_ctrl, N34_detr_des_ctrl)
plt.scatter(N12_detr_des_rcp , N34_detr_des_rcp )

# lagges correlation
plt.figure()
plt.axhline(0, c='k', lw=.5)
plt.axvline(0, c='k', lw=.5)
for l in np.arange(-15,16,1):
    if l<0:
        plt.scatter(l, np.corrcoef(N12_detr_des_ctrl[:l], N34_detr_des_ctrl.shift(time=l).dropna(dim='time'))[0,1], c='C0')
        plt.scatter(l, np.corrcoef(N12_detr_des_rcp [:l], N34_detr_des_rcp .shift(time=l).dropna(dim='time'))[0,1], c='C1', marker='D')
    else:
        plt.scatter(l, np.corrcoef(N12_detr_des_ctrl[l:], N34_detr_des_ctrl.shift(time=l).dropna(dim='time'))[0,1], c='C0')
        plt.scatter(l, np.corrcoef(N12_detr_des_rcp [l:], N34_detr_des_rcp .shift(time=l).dropna(dim='time'))[0,1], c='C1', marker='D')

# Atlantic Ocean

## Atlantic Multidecadal Oscillation
from Wikipedia:
> Several methods have been proposed to remove the global trend and El Niño-Southern Oscillation (ENSO) influence over the North Atlantic SST. Trenberth and Shea, assuming that the effect of global forcing over the North Atlantic is similar to the global ocean, subtracted the global (60°N-60°S) mean SST from the North Atlantic SST to derive a revised AMO index.[6]
>
> Ting et al. however argue that the forced SST pattern is not globally uniform; they separated the forced and internally generated variability using signal to noise maximizing EOF analysis.[2]
>
> Van Oldenborgh et al. derived an AMO index as the SST averaged over the extra-tropical North Atlantic (to remove the influence of ENSO that is greater at tropical latitude) minus the regression on global mean temperature.[7]
>
> Guan and Nigam removed the non stationary global trend and Pacific natural variability before applying an EOF analysis to the residual North Atlantic SST.[8]
>
> The linearly detrended index suggests that the North Atlantic SST anomaly at the end of the twentieth century is equally divided between the externally forced component and internally generated variability, and that the current peak is similar to middle twentieth century; by contrast the others methodology suggest that a large portion of the North Atlantic anomaly at the end of the twentieth century is externally forced.[2]

We use the method of Trenberth here: averaged SST [0-60 N]x[0-80W] - avg. SST [60S-60N], then .

In [ ]:
# %%time
# # 30 sec
# TAREA_AMO   = TAREA.sel(AMO_area).sum()
# TAREA_T_exT = TAREA.sel(T_exT_area).sum()
# NA_ctrl     = SST_index(xa_SST=SST_yrly_ctrl, AREA=TAREA, index_loc=AMO_area  , AREA_index=TAREA_AMO  , MASK=MASK_ocn, dims=('nlat', 'nlon'))
# T_exT_ctrl  = SST_index(xa_SST=SST_yrly_ctrl, AREA=TAREA, index_loc=T_exT_area, AREA_index=TAREA_T_exT, MASK=MASK_ocn, dims=('nlat', 'nlon'))
# NA_rcp      = SST_index(xa_SST=SST_yrly_rcp , AREA=TAREA, index_loc=AMO_area  , AREA_index=TAREA_AMO  , MASK=MASK_ocn, dims=('nlat', 'nlon'))
# T_exT_rcp   = SST_index(xa_SST=SST_yrly_rcp , AREA=TAREA, index_loc=T_exT_area, AREA_index=TAREA_T_exT, MASK=MASK_ocn, dims=('nlat', 'nlon'))
# AMO_ctrl    = (NA_ctrl - T_exT_ctrl) - (NA_ctrl - T_exT_ctrl).mean()
# AMO_rcp     = (NA_rcp  - T_exT_rcp ) - (NA_rcp  - T_exT_rcp ).mean()
# AMO_ctrl.to_netcdf(f'{path_results}/SST/AMO_yrly_ctrl.nc')
# AMO_rcp .to_netcdf(f'{path_results}/SST/AMO_yrly_rcp.nc' )

In [ ]:
AMO_ctrl = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_ctrl.nc', decode_times=False)
AMO_rcp  = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_rcp.nc' , decode_times=False)

In [ ]:
f, ax = plt.subplots(1,1,figsize=(8,5))
ax.tick_params(labelsize=14)
ax.axhline(0,c='k', lw=.5)
ax.axvline(200,c='g', lw=.5)
ax.plot(AMO_ctrl.time/365     , AMO_ctrl, c='C0', lw=.5)
ax.plot(AMO_rcp .time/365-1800, AMO_rcp , c='C1', lw=.5)
L1 = ax.plot(AMO_ctrl.time/365     , AMO_ctrl.rolling(time=15, center=True).mean(), c='C0', lw=2, label='CTRL')
L2 = ax.plot(AMO_rcp .time/365-1800, AMO_rcp .rolling(time=15, center=True).mean(), c='C1', lw=2, label='RCP' )
ax.text(.02, .02, '15 year running mean', transform=ax.transAxes, fontsize=14, color='grey')
ax.legend(loc=2, ncol=2, fontsize=14)
ax.set_xlabel('time [years]' , fontsize=16)
ax.set_ylabel('AMO index [K]', fontsize=16)
f.tight_layout()
plt.savefig(f'{path_results}/SST/AMO_index')

- comparable period to observations (approx 70 years; Trenberth & Shea)
- much smaller amplitude though (0.05K vs. 0.2K)